In [73]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
# soup = BeautifulSoup(open(r'classification_codes_data\arxiv_category_taxonomy.html'), 'html.parser')
# classification_codes = soup.get_text().split('\n')
# classification_codes = [code for code in classification_codes if (code != '' and code not in main_cls)]
# classification_codes = [code for code in classification_codes if (code != '')]
# classification_codes = pd.DataFrame(classification_codes, columns=['classification_codes'])
# classification_codes.to_csv(r'arxiv_classification.txt', index=False)
# df['code'] = df['class'].apply(lambda x: x.split(' '))
# df['class_title'] = df['code'].apply(lambda x: ' '.join(x[1:]))
# df['class_code'] = df['code'].apply(lambda x: x[0])
# df.drop(['code','class'], axis=1, inplace=True)
# df.to_csv(r'arxiv_classification.csv', index=False)

In [ ]:
# USE THE CSV FILE FROM NOW ON

In [107]:
df = pd.read_csv(r'arxiv_classification.csv')
identifiers = df['class_code'].values
identifiers = [identifier.split('.')[0] for identifier in identifiers]
identifiers = list(dict.fromkeys(identifiers))
identifiers

['cs',
 'econ',
 'eess',
 'math',
 'astro-ph',
 'cond-mat',
 'gr-qc',
 'hep-ex',
 'hep-lat',
 'hep-ph',
 'hep-th',
 'math-ph',
 'nlin',
 'nucl-ex',
 'nucl-th',
 'physics',
 'quant-ph',
 'q-bio',
 'q-fin',
 'stat']

In [108]:
classes = ["Computer Science", "Economics", "Electrical Engineering and Systems Science", "Mathematics", "Astrophyics", 
           "Condensed Matter", "General Relativity and Quantum Cosmology", "High Energy Physics - Experiment", 
           "High Energy Physics - Lattice", "High Energy Physics - Phenomenology", "High Energy Physics - Theory",
           "Mathematical Physics", "Non-linear Sciences", "Nuclear Experiment", "Nuclear Theory", "Physics",
           "Quantum Physics", "Quantitative Biology", "Quantitative Finance", "Statistics"]

In [109]:
assert len(classes) == len(identifiers)

In [110]:
class_identifiers = list(zip(identifiers, classes))
class_identifiers = pd.DataFrame(class_identifiers, columns=['class_code', 'class_title'])
class_identifiers

,class_code,class_title
0,cs,Computer Science
1,econ,Economics
2,eess,Electrical Engineering and Systems Science
3,math,Mathematics
4,astro-ph,Astrophyics
5,cond-mat,Condensed Matter
6,gr-qc,General Relativity and Quantum Cosmology
7,hep-ex,High Energy Physics - Experiment
8,hep-lat,High Energy Physics - Lattice
9,hep-ph,High Energy Physics - Phenomenology


In [112]:
df['class_title'] = df['class_title'].apply(lambda x: x.replace('(','').replace(')',''))

In [113]:
df['identifier'] = df['class_code'].apply(lambda x: x.split('.'))

In [114]:
df

,description,class_title,class_code,identifier
0,Covers all areas of AI except Vision Roboti...,Artificial Intelligence,cs.AI,"[cs, AI]"
1,Covers systems organization and hardware arc...,Hardware Architecture,cs.AR,"[cs, AR]"
2,Covers models of computation complexity cla...,Computational Complexity,cs.CC,"[cs, CC]"
3,Covers applications of computer science to t...,Computational Engineering Finance and Science,cs.CE,"[cs, CE]"
4,Roughly includes material in ACM Subject Cla...,Computational Geometry,cs.CG,"[cs, CG]"
...,...,...,...,...
150,Algorithms Simulation Visualization,Computation,stat.CO,"[stat, CO]"
151,Design Surveys Model Selection Multiple T...,Methodology,stat.ME,"[stat, ME]"
152,Covers machine learning papers (supervised ...,Machine Learning,stat.ML,"[stat, ML]"
153,Work in statistics that does not fit into th...,Other Statistics,stat.OT,"[stat, OT]"


In [137]:
df_acm = pd.read_csv(r'acm_classification.csv')
df_acm[df_acm['class'] == 'H.1.2'].values[0][1]

'Class: Information systems Subclass: Models and principles Subsubclass: User/machine systems'

In [139]:
import re

In [195]:
def construct_full_text(identifier):
    if isinstance(identifier, str):
        identifier = identifier.split('.')

    code = '.'.join(identifier)

    if code not in df['class_code'].values:
        raise ValueError('Code not found in the dataset')

    description = list(df[df['class_code'] == code].values[0])[0].strip().replace('  ', ', ')
    cls = class_identifiers[class_identifiers['class_code'] == identifier[0]].values[0][1]
    sub_cls = list(df[df['class_code'] == code].values[0])[1].strip()

    acm = re.findall(r'[A-Z].[0-9].[0-9]', description)

    if acm:
        acm_descriptions = []
        
        for a in acm:
            acm_descriptions.append(df_acm[df_acm['class'] == a].values[0][1])
        acm_descriptions = ', '.join(acm_descriptions)

        description = re.sub(r'. Roughly includes material in ACM Subject Classes.+\.\s', ' ', description)

        full_text = f'Class: {cls} Subclass: {sub_cls} Description: {description} ALSO {acm_descriptions}'
        full_text = re.sub(r'Roughly includes material in ACM Subject Classes.+\.\s', '', full_text)

        return full_text
    else:
        full_text = f'Class: {cls} Subclass: {sub_cls} Description: {description}'
        return full_text
        
    

In [196]:
df['full_text'] = df['identifier'].apply(construct_full_text)

In [197]:
print(df['full_text'].values)

['Class: Computer Science Subclass: Artificial Intelligence Description: Covers all areas of AI except Vision, Robotics, Machine Learning, Multiagent Systems, and Computation and Language (Natural Language Processing), which have separate subject areas. In particular, includes Expert Systems, Theorem Proving (although this may overlap with Logic in Computer Science), Knowledge Representation, Planning, and Uncertainty in AI. ALSO Class: Computing methodologies Subclass: Artificial intelligence Subsubclass: General, Class: Computing methodologies Subclass: Artificial intelligence Subsubclass: Applications and expert systems, Class: Computing methodologies Subclass: Artificial intelligence Subsubclass: Deduction and theorem proving, Class: Computing methodologies Subclass: Artificial intelligence Subsubclass: Knowledge representation formalisms and methods, Class: Computing methodologies Subclass: Artificial intelligence Subsubclass: Problem solving, control methods, and search, Class: C